In [39]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [40]:
import addict
import datetime
import os
import pyproj
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from okada_wrapper import dc3dwrapper

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [41]:
import celeri
celeri = reload(celeri)

In [42]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
# command_file_name = './data/global/global_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

In [43]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)

Isolating slip rate constraints
- Dip-slip rate constraint on Nevada02abb: rate = -2.40 (mm/yr), 1-sigma uncertainty = +/-1.20 (mm/yr)
- Strike-slip rate constraint on SierraConnectbb: rate = -1.70 (mm/yr), 1-sigma uncertainty = +/-0.80 (mm/yr)
- Strike-slip rate constraint on TestSjfa: rate = -17.50 (mm/yr), 1-sigma uncertainty = +/-8.80 (mm/yr)
- Strike-slip rate constraint on aaa: rate = -3.50 (mm/yr), 1-sigma uncertainty = +/-1.70 (mm/yr)
- Strike-slip rate constraint on asdf: rate = -4.00 (mm/yr), 1-sigma uncertainty = +/-2.00 (mm/yr)
- Strike-slip rate constraint on asdf: rate = 2.80 (mm/yr), 1-sigma uncertainty = +/-2.50 (mm/yr)
- Strike-slip rate constraint on asdfb: rate = -9.20 (mm/yr), 1-sigma uncertainty = +/-4.60 (mm/yr)
- Strike-slip rate constraint on asdfba: rate = -17.00 (mm/yr), 1-sigma uncertainty = +/-8.50 (mm/yr)
- Dip-slip rate constraint on asdfbb: rate = -3.00 (mm/yr), 1-sigma uncertainty = +/-1.50 (mm/yr)
- Strike-slip rate constraint on cfm_camp_rock_complete_

In [44]:
def fault_parameters_to_okada_format(sx1, sy1, sx2, sy2, dip, D, bd):
    """
    This function takes fault trace, dip, and locking depth information
    and calculates the anchor coordinates, length, width and strike of
    the fault plane following (1985).

    Arguments:
        sx1 : x coord of fault trace endpoint 1
        sy1 : y coord of fault trace endpoint 1
        sx2 : x coord of fault trace endpoint 2
        sy2 : y coord of fault trace endpoint 2
        dip : dip of fault plane (degrees)
        D : fault locking depth
        bd : burial depth (top "locking depth")

    Returned variables:
        strike : stike of fault plane
        L : fault length
        W : fault width
        ofx : x coord of fault anchor
        ofy : y coord of fault anchor
        ofxe : x coord of other buried corner
        ofye : y coord of other buried corner
        tfx : x coord of fault anchor (top relative)
        tfy : y coord of fault anchor (top relative)
        tfxe : x coord of other buried corner (top relative)
        tfye : y coord of other buried corner (top relative)
    """

    okada_parameters = addict.Dict()
    okada_parameters.strike = np.arctan2(sy1 - sy2, sx1 - sx2) + np.pi # This is by convention
    okada_parameters.L = np.sqrt((sx2 - sx1)**2 + (sy2 - sy1)**2)
    okada_parameters.W = (D - bd) / np.sin(np.deg2rad(dip))

    # Calculate fault segment anchor and other buried point
    okada_parameters.ofx = sx1 + D / np.tan(np.deg2rad(dip)) * np.sin(np.deg2rad(okada_parameters.strike))
    okada_parameters.ofy = sy1 - D / np.tan(np.deg2rad(dip)) * np.cos(np.deg2rad(okada_parameters.strike))
    okada_parameters.ofxe = sx2 + D / np.tan(np.deg2rad(dip)) * np.sin(np.deg2rad(okada_parameters.strike))
    okada_parameters.ofye = sy2 - D / np.tan(np.deg2rad(dip)) * np.cos(np.deg2rad(okada_parameters.strike))

    # Calculate fault segment anchor and other buried point (top relative)
    okada_parameters.tfx = sx1 + bd / np.tan(np.deg2rad(dip)) * np.sin(np.deg2rad(okada_parameters.strike))
    okada_parameters.tfy = sy1 - bd / np.tan(np.deg2rad(dip)) * np.cos(np.deg2rad(okada_parameters.strike))
    okada_parameters.tfxe = sx2 + bd / np.tan(np.deg2rad(dip)) * np.sin(np.deg2rad(okada_parameters.strike))
    okada_parameters.tfye = sy2 - bd / np.tan(np.deg2rad(dip)) * np.cos(np.deg2rad(okada_parameters.strike))

    return okada_parameters


# def GetElasticPartials(segment, stations):
"""
Calculates the elastic displacement partial derivatives based on the Okada
formulation, using the source and receiver geometries defined in
dicitonaries segment and stations. Before calculating the partials for
each segment, a local oblique Mercator project is done.
"""

n_stations = len(station)
n_segments = len(segment)
G = np.zeros((3 * n_stations, 3 * n_segments))
v1 = np.zeros(n_segments)
v2 = np.zeros(n_segments)
v3 = np.zeros(n_segments)

# Loop through each segment and calculate displacements
# for i in range(n_segments):
for i in range(1):
    print(i)

    # Local map projection
    projection = celeri.get_segment_oblique_projection(segment.lon1[i], segment.lat1[i], segment.lon2[i], segment.lat2[i])
    station_x, station_y = projection(station.lon, station.lat)
    segment_x1, segment_y1 = projection(segment.lon1[i], segment.lat1[i])
    segment_x2, segment_y2 = projection(segment.lon2[i], segment.lat2[i])

    # Calculate fault parameters in Okada form
    # okada_parameters = fault_parameters_to_okada_format(f.px1, f.py1, f.px2, f.py2, segment.dip[i], segment.locking_depth[i], segment.burial_depth[i])
    okada_parameters = fault_parameters_to_okada_format(segment_x1, segment_y1,
        segment_x2, segment_y2, segment.dip[i], segment.locking_depth[i], 
        segment.burial_depth[i])

    # Translate observation coordinates relative to fault anchor

    # Rotation observation coordinates to remove strike.
    # rot = np.array([[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])
    rotation_matrix = np.array([[np.cos(np.deg2rad(okada_parameters.strike)), -np.sin(np.deg2rad(okada_parameters.strike))], 
                                [np.sin(np.deg2rad(okada_parameters.strike)), np.cos(np.deg2rad(okada_parameters.strike))]])

#     % Displacements due to unit slip components
#     [ves vns vus...
#      ved vnd vud...
#      vet vnt vut] = okada_partials(ofx, ofy, strike, f.lDep, deg_to_rad(f.dip), L, W, 1, 1, 1, s.fpx, s.fpy, command.poissons_ratio);
     
#     v1{i} = reshape([ves vns vus]', 3*nSta, 1)
#     v2{i} = reshape(sign(90 - f.dip).*[ved vnd vud]', 3*nSta, 1)
#     v3{i} = reshape((f.dip - 90 == 0).*[vet vnt vut]', 3*nSta, 1) 
#     v1{i} = xyz2enumat((v1{i}), -f.strike + 90)
#     v2{i} = xyz2enumat((v2{i}), -f.strike + 90)
#     v3{i} = xyz2enumat((v3{i}), -f.strike + 90)

# Place cell arrays into the partials matrix
# G[:, 0::3] = cell2mat(v1)
# G[:, 1::3] = cell2mat(v2)
# G[:, 2::3] = cell2mat(v3) 


0


In [49]:
# TODO: Locking depths are currently meters rather than KM in inputfiles!!!
# TODO: Is there another XYZ to ENU conversion needed?

i = 0
segment.locking_depth.values[i] *= celeri.KM2M
segment.burial_depth.values[i] *= celeri.KM2M

# Project coordinates to flat space using a local oblique Mercator projection
projection = celeri.get_segment_oblique_projection(segment.lon1[i], segment.lat1[i], segment.lon2[i], segment.lat2[i], True)
station_x, station_y = projection(station.lon, station.lat)
segment_x1, segment_y1 = projection(segment.lon1[i], segment.lat1[i])
segment_x2, segment_y2 = projection(segment.lon2[i], segment.lat2[i])

# Calculate geometric fault parameters
print(type(segment_x1), segment_x1)
print(type(segment_y1), segment_y1)
print(type(segment_x2), segment_x2)
print(type(segment_y2), segment_y2)
segment_strike = np.arctan2(segment_y2 - segment_y1, segment_x2 - segment_x1) # radians
print(type(segment_strike), segment_strike)

segment_length = np.sqrt((segment_y2 - segment_y1)**2.0 + (segment_x2 - segment_x1)**2.0)
segment_up_dip_width = (segment.locking_depth[i] - segment.burial_depth[i]) / np.sin(np.deg2rad(segment.dip[i]))

# Translate stations and segment so that segment mid-point is at the origin
segment_x_mid = (segment_x1 + segment_x2) / 2.0
segment_y_mid = (segment_y1 + segment_y2) / 2.0
station_x -= segment_x_mid
station_y -= segment_y_mid
segment_x1 -= segment_x_mid
segment_x2 -= segment_x_mid
segment_y1 -= segment_y_mid
segment_y2 -= segment_y_mid

# Unrotate coordinates to eliminate strike, segment will lie along y = 0
# np.einsum guidance from: https://stackoverflow.com/questions/26289972/use-numpy-to-multiply-a-matrix-across-an-array-of-points
rotation_matrix = np.array([[np.cos(segment_strike), -np.sin(segment_strike)], 
                            [np.sin(segment_strike), np.cos(segment_strike)]])
un_rotation_matrix = np.array([[np.cos(-segment_strike), -np.sin(-segment_strike)], 
                               [np.sin(-segment_strike), np.cos(-segment_strike)]])
station_x_rotated, station_y_rotated = np.hsplit(np.einsum("ij,kj->ik", np.dstack((station_x, station_y))[0], un_rotation_matrix), 2)
segment_x1_rotated, segment_y1_rotated = un_rotation_matrix.dot([segment_x1, segment_y1])
segment_x2_rotated, segment_y2_rotated = un_rotation_matrix.dot([segment_x2, segment_y2])

# print(station_x.shape, type(station_x))
# print(station_y.shape, type(station_y))
# print(un_rotation_matrix.shape, un_rotation_matrix)
# print(type(un_rotation_matrix))
# print(type(segment_strike), segment_strike)



# Elastic displacements due to fault slip from Okada 1985
alpha = (command.material_lambda + command.material_mu) / (command.material_lambda + 2 * command.material_mu)
u_x = np.zeros_like(station_x)
u_y = np.zeros_like(station_x)
u_z = np.zeros_like(station_x)
for j in range(len(station)):
    _, u, _ = dc3dwrapper(alpha, # (lambda + mu) / (lambda + 2 * mu)
                          [station_x_rotated[j], station_y_rotated[j], 0], # (meters) observation point
                          -segment.locking_depth[i], # (meters) depth of the fault origin
                          segment.dip[i], # (degrees) the dip-angle of the rectangular dislocation surface
                          [-segment_length / 2, segment_length / 2], # (meters) the along-strike range of the surface (al1,al2 in the original)
                          [0, segment_up_dip_width], # (meters) along-dip range of the surface (aw1, aw2 in the original)
                          [1.0, 0.0, 0.0]) # (meters) strike-slip, dip-slip, tensile-slip
    u_x[j] = u[0]
    u_y[j] = u[1]
    u_z[j] = u[2]


# Un-rotate displacement to account for projected fault strike
u_x_un_rotated, u_y_un_rotated = np.hsplit(np.einsum("ij,kj->ik", np.dstack((u_x, u_y))[0], rotation_matrix), 2)



# plt.figure()
# plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-r")
# plt.plot(station.lon, station.lat, '.b', markersize=1)
# plt.xlim([235, 255])
# plt.ylim([30, 50])
# plt.gca().set_aspect('equal', adjustable='box')
# plt.title("Positions: longitude and latitude")
# plt.show()

# plt.figure()
# plt.plot([segment_x1, segment_x2], [segment_y1, segment_y2], "-r")
# plt.plot(station_x, station_y, '.b', markersize=1)
# plt.xlim([-1e6, 1e6])
# plt.ylim([-1e6, 1e6])
# plt.gca().set_aspect('equal', adjustable='box')
# plt.title("Positions: projected and translated")
# plt.show()

# plt.figure()
# plt.plot([segment_x1_rotated, segment_x2_rotated], [segment_y1_rotated, segment_y2_rotated], "-r")
# plt.plot(station_x_rotated, station_y_rotated, '.b', markersize=1)
# plt.xlim([-1e6, 1e6])
# plt.ylim([-1e6, 1e6])
# plt.gca().set_aspect('equal', adjustable='box')
# plt.title("Positions: projected, translated, and rotated")
# plt.show()

# plt.figure()
# plt.plot([segment_x1_rotated, segment_x2_rotated], [segment_y1_rotated, segment_y2_rotated], "-r")
# # plt.plot(station_x_rotated, station_y_rotated, '.b', markersize=1)
# plt.quiver(station_x_rotated, station_y_rotated, u_x, u_y, scale=1e-1, scale_units='inches')
# plt.xlim([-1e6, 1e6])
# plt.ylim([-1e6, 1e6])
# plt.gca().set_aspect('equal', adjustable='box')
# plt.title("Displacements: projected, translated, and rotated")
# plt.show()

# plt.figure()
# plt.plot([segment_x1, segment_x2], [segment_y1, segment_y2], "-r")
# # plt.plot(station_x, station_y, '.b', markersize=1)
# plt.quiver(station_x, station_y, u_x_un_rotated, u_y_un_rotated, scale=1e-1, scale_units='inches')
# plt.xlim([-1e6, 1e6])
# plt.ylim([-1e6, 1e6])
# plt.gca().set_aspect('equal', adjustable='box')
# plt.title("Displacements: projected and translated")
# plt.show()

# plt.figure()
# plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-r")
# # plt.plot(station_x, station_y, '.b', markersize=1)
# plt.quiver(station.lon, station.lat, u_x_un_rotated, u_y_un_rotated, scale=1e-1, scale_units='inches')
# plt.xlim([235, 255])
# plt.ylim([30, 50])
# plt.gca().set_aspect('equal', adjustable='box')
# plt.title("Displacements: longitude and latitude")
# plt.show()




<class 'float'> -1093573.9280795865
<class 'float'> 4890861.787021328
<class 'float'> -1053279.8319317077
<class 'float'> 4710651.880738813
<class 'numpy.float64'> -1.3508192569114943


In [55]:
def get_okada_displacements(segment_lon1, segment_lat1, segment_lon2, segment_lat2, segment_locking_depth, segment_burial_depth, segment_dip, material_lambda, material_mu, station_lon, station_lat):
    """
    TODO: Locking depths are currently meters rather than KM in inputfiles!!!
    TODO: Is there another XYZ to ENU conversion needed?
    """
    segment_locking_depth *= celeri.KM2M
    segment_burial_depth *= celeri.KM2M

    # Project coordinates to flat space using a local oblique Mercator projection
    projection = celeri.get_segment_oblique_projection(segment_lon1, segment_lat1, segment_lon2, segment_lat2)
    station_x, station_y = projection(station_lon, station_lat)
    segment_x1, segment_y1 = projection(segment_lon1, segment_lat1)
    segment_x2, segment_y2 = projection(segment_lon2, segment_lat2)

    # Calculate geometric fault parameters
    segment_strike = np.arctan2(segment_y2 - segment_y1, segment_x2 - segment_x1) # radians
    segment_length = np.sqrt((segment_y2 - segment_y1)**2.0 + (segment_x2 - segment_x1)**2.0)
    segment_up_dip_width = (segment_locking_depth - segment_burial_depth) / np.sin(np.deg2rad(segment_dip))

    # Translate stations and segment so that segment mid-point is at the origin
    segment_x_mid = (segment_x1 + segment_x2) / 2.0
    segment_y_mid = (segment_y1 + segment_y2) / 2.0
    station_x -= segment_x_mid
    station_y -= segment_y_mid
    segment_x1 -= segment_x_mid
    segment_x2 -= segment_x_mid
    segment_y1 -= segment_y_mid
    segment_y2 -= segment_y_mid

    # Unrotate coordinates to eliminate strike, segment will lie along y = 0
    rotation_matrix = np.array([[np.cos(segment_strike), -np.sin(segment_strike)], 
                                [np.sin(segment_strike), np.cos(segment_strike)]])
    un_rotation_matrix = np.array([[np.cos(-segment_strike), -np.sin(-segment_strike)], 
                                [np.sin(-segment_strike), np.cos(-segment_strike)]])
    station_x_rotated, station_y_rotated = np.hsplit(np.einsum("ij,kj->ik", np.dstack((station_x, station_y))[0], un_rotation_matrix), 2)

    # Elastic displacements due to fault slip from Okada 1985
    # alpha = (command.material_lambda + command.material_mu) / (command.material_lambda + 2 * command.material_mu)
    alpha = (material_lambda + material_mu) / (material_lambda + 2 * material_mu)
    u_x = np.zeros_like(station_x)
    u_y = np.zeros_like(station_x)
    u_up = np.zeros_like(station_x)
    for i in range(len(station)):
        _, u, _ = dc3dwrapper(alpha, # (lambda + mu) / (lambda + 2 * mu)
                            [station_x_rotated[i], station_y_rotated[i], 0], # (meters) observation point
                            -segment_locking_depth, # (meters) depth of the fault origin
                            segment_dip, # (degrees) the dip-angle of the rectangular dislocation surface
                            [-segment_length / 2, segment_length / 2], # (meters) the along-strike range of the surface (al1,al2 in the original)
                            [0, segment_up_dip_width], # (meters) along-dip range of the surface (aw1, aw2 in the original)
                            [1.0, 0.0, 0.0]) # (meters) strike-slip, dip-slip, tensile-slip
        u_x[i] = u[0]
        u_y[i] = u[1]
        u_up[i] = u[2]

    # Un-rotate displacement to account for projected fault strike
    u_east, u_north = np.hsplit(np.einsum("ij,kj->ik", np.dstack((u_x, u_y))[0], rotation_matrix), 2)
    return u_east, u_north, u_up

i = 0
u_east, u_north, u_up = get_okada_displacements(segment.lon1.values[i], segment.lat1[i], segment.lon2[i], segment.lat2[i], segment.locking_depth[i], segment.burial_depth[i], segment.dip[i], command.material_lambda, command.material_mu, station.lon, station.lat)

<class 'numpy.float64'> 184659.75334849526


In [ ]:
station.lon.values

In [ ]:
segment_strike.item()